In [8]:
import scri
import numpy as np
import sxs
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

In [9]:
# Download the latest catalog
catalog = sxs.load("catalog",download = False)
# Do some query, this is a pandas dataframe:
ctable = catalog.table
BBHs = ctable[ctable['object_types'] == 'BHBH']

In [10]:
BBHs

,object_types,initial_separation,initial_orbital_frequency,initial_adot,initial_ADM_energy,initial_ADM_linear_momentum,initial_ADM_linear_momentum_mag,initial_ADM_angular_momentum,initial_ADM_angular_momentum_mag,initial_mass1,...,remnant_dimensionless_spin,remnant_dimensionless_spin_mag,remnant_velocity,remnant_velocity_mag,eos,initial_data_type,disk_mass,ejecta_mass,url,metadata_path
SXS:BBH:0001,BHBH,18.000000,0.012203,-0.000025,0.993736,"[7.5160071e-09, 2.4389069e-09, -5.798632e-10]",7.923059e-09,"[4.27581822e-08, -2.342625073e-07, 1.184715566...",1.184716,0.50,...,"[-7.29520687012e-10, 7.40468371215e-10, 0.6864...",0.686462,"[1.45734870604e-09, 7.51603528474e-10, 3.72053...",4.065850e-09,NaN,BBH_CFMS,NaN,NaN,https://doi.org/10.5281/zenodo.1212426,SXS:BBH:0001v6/Lev5/metadata.json
SXS:BBH:0002,BHBH,19.000000,0.011292,-0.000021,0.994031,"[1.6810827e-09, 1.4244131e-09, -1.14839759e-08]",1.169345e-08,"[2.7620419806e-06, -2.4287931932e-06, 1.210130...",1.210131,0.50,...,"[3.75218798083e-10, 1.32614402018e-09, 0.68644...",0.686448,"[3.59963486031e-09, 7.18141003028e-09, 1.18947...",1.435326e-08,NaN,BBH_CFMS,NaN,NaN,https://doi.org/10.5281/zenodo.1210580,SXS:BBH:0002v7/Lev6/metadata.json
SXS:BBH:0003,BHBH,19.000000,0.011284,-0.000019,0.994277,"[2.599703e-09, -4.1239227e-09, 6.03918869017e-05]",6.039189e-05,"[0.1251774015530911, -0.0006755685628109, 1.21...",1.216661,0.50,...,"[0.0664601205164, 0.00957605779857, 0.69144717...",0.694700,"[0.000404512999056, 2.21067123662e-05, 0.00229...",2.328050e-03,NaN,BBH_CFMS,NaN,NaN,https://doi.org/10.5281/zenodo.1212572,SXS:BBH:0003v5/Lev5/metadata.json
SXS:BBH:0004,BHBH,19.000000,0.011313,-0.000028,0.994314,"[-2.7524803e-09, 5.883293e-09, 5.3376578e-09]",8.407132e-09,"[8.163645183e-07, 7.736321355e-07, 1.094457487...",1.094457,0.50,...,"[9.13765586478e-10, -3.77301001538e-10, 0.6082...",0.608209,"[0.000272458957071, 0.000295925023502, 7.01870...",4.022505e-04,NaN,BBH_CFMS,NaN,NaN,https://doi.org/10.5281/zenodo.1212574,SXS:BBH:0004v5/Lev6/metadata.json
SXS:BBH:0005,BHBH,18.000000,0.012168,-0.000016,0.993947,"[-3.7413651e-09, -2.4006761e-09, 7.874335e-10]",4.514544e-09,"[9.790607752e-07, 1.026494432e-07, 1.300909689...",1.300910,0.50,...,"[-3.59606685604e-11, -4.66653370904e-09, 0.760...",0.760698,"[-8.7434463379e-05, -0.000333724306139, 1.9559...",3.449880e-04,NaN,BBH_CFMS,NaN,NaN,https://doi.org/10.5281/zenodo.1212580,SXS:BBH:0005v5/Lev5/metadata.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SXS:BBH:2160,BHBH,14.950875,0.015675,-0.000015,0.994386,"[-4.3017867e-09, -5.84941013e-08, -2.6828e-12]",5.865207e-08,"[-1.72014e-11, 1.12e-11, 1.1265748937101803]",1.126575,0.75,...,"[-1.33852786528e-08, 1.75125212483e-07, 0.7854...",0.785484,"[-0.000226736030367, -1.15891313373e-06, 1.244...",2.267390e-04,NaN,BBH_SKS,NaN,NaN,https://doi.org/10.5281/zenodo.2631184,SXS:BBH:2160v4/Lev4/metadata.json
SXS:BBH:2161,BHBH,14.964217,0.015690,0.000039,0.994372,"[-1.89327e-10, 1.975285e-10, 4.71702e-11]",2.776459e-10,"[1.50934e-11, -1.5467e-12, 1.1490226675264523]",1.149023,0.75,...,"[5.57348382385e-07, 2.40198000437e-07, 0.79640...",0.796406,"[-0.000129002712695, 0.000166590631856, 1.8561...",2.106992e-04,NaN,BBH_SKS,NaN,NaN,https://doi.org/10.5281/zenodo.2631186,SXS:BBH:2161v4/Lev4/metadata.json
SXS:BBH:2162,BHBH,14.867169,0.015691,0.000119,0.994329,"[-1.5122701e-09, -3.59595772e-08, -2.8187e-12]",3.599136e-08,"[-2.19336e-11, 7.8297e-12, 1.1692072355385335]",1.169207,0.75,...,"[-4.89070843408e-08, 7.6437110503e-09, 0.80696...",0.806964,"[2.61163132178e-05, -0.000188716649709, 2.1175...",1.905152e-04,NaN,BBH_SKS,NaN,NaN,https://doi.org/10.5281/zenodo.2631188,SXS:BBH:2162v4/Lev4/metadata.json
SXS:BBH:2163,BHBH,14.758416,0.015696,0.000195,0.994295,"[-2.18530817e-08, -3.88038064e-08, -2.90331e-11]",4.453418e-08,"[2.7331e-11, 6.0788e-12, 1.177987391087692]",1.177987,0.75,...,"[2.3947276397e-08, 7.77942466837e-08, 0.811977...",0.811977,"[-6.19263098699e-05, -0.000143640

In [12]:
system='SXS:BBH:0389'

In [13]:
md = sxs.load(system + "/Lev/metadata.json")
W = sxs.load(system + "/Lev/rhOverM", extrapolation_order=4)

Skipping download from 'https://data.black-holes.org/catalog.json' because local file is newer
Found the following files to load from the SXS catalog:
    SXS:BBH:0389v5/Lev3/metadata.json
Found the following files to load from the SXS catalog:
    SXS:BBH:0389v5/Lev3/rhOverM_Asymptotic_GeometricUnits_CoM.h5


In [4]:
abd = scri.asymptotic_bondi_data.AsymptoticBondiData(
    time = np.linspace(0,10,11),
    ell_max = 8,
    multiplication_truncator = max,)